# Module 1: Data Loading and Augmentation Using Keras

In [ ]:
import os, glob, numpy as np, matplotlib.pyplot as plt
from PIL import Image, ImageDraw

DATASET_DIR = "./images_dataSAT"
DIR_NON_AGRI = os.path.join(DATASET_DIR, "class_0_non_agri")
DIR_AGRI = os.path.join(DATASET_DIR, "class_1_agri")

def _ensure_dataset():
    os.makedirs(DIR_NON_AGRI, exist_ok=True)
    os.makedirs(DIR_AGRI, exist_ok=True)
    if len(os.listdir(DIR_NON_AGRI))>0 and len(os.listdir(DIR_AGRI))>0:
        return
    import numpy as np
    from PIL import Image, ImageDraw
    rng = np.random.default_rng(0)
    for cls_dir, pattern in [(DIR_NON_AGRI, 'rect'), (DIR_AGRI, 'lines')]:
        for i in range(12):
            img = Image.new("RGB",(64,64),(rng.integers(20,235),rng.integers(20,235),rng.integers(20,235)))
            d = ImageDraw.Draw(img)
            if pattern=='rect':
                d.rectangle([10,10,54,54], outline=(255,255,255), width=2)
            else:
                for y in range(5,64,10):
                    d.line([0,y,64,y], fill=(255,255,255), width=1)
            img.save(os.path.join(cls_dir, f"img_{{i:03d}}.png"))

# Copy dataset from /mnt/data if available
if os.path.exists('/mnt/data/images_dataSAT'):
    import shutil
    if not os.path.exists(DATASET_DIR):
        shutil.copytree('/mnt/data/images_dataSAT', DATASET_DIR)
_ensure_dataset()
print("Dataset ready at", os.path.abspath(DATASET_DIR))

In [ ]:
import os, glob
all_image_paths = sorted(glob.glob(os.path.join(DATASET_DIR, '*', '*.png')))
print("Total images:", len(all_image_paths))
print("First 5 paths:", all_image_paths[:5])

In [ ]:
def path_to_label(p):
    return 1 if "class_1_agri" in p else 0
temp = list(zip(all_image_paths, [path_to_label(p) for p in all_image_paths]))
print("Sample 5:", temp[:5])

In [ ]:
# Custom generator batch size 8
import numpy as np
from PIL import Image
def custom_data_generator(pairs, batch_size=8, shuffle=True):
    idxs = np.arange(len(pairs))
    while True:
        if shuffle: np.random.shuffle(idxs)
        for s in range(0, len(idxs), batch_size):
            b = idxs[s:s+batch_size]
            X, y = [], []
            for i in b:
                p, lab = pairs[i]
                X.append(np.array(Image.open(p).resize((64,64)))/255.0); y.append(lab)
            yield np.array(X), np.array(y)
gen = custom_data_generator(temp, batch_size=8)
Xb, yb = next(gen)
print("Batch:", Xb.shape, yb.shape)

In [ ]:
# Validation data with batch 8 using Keras
import tensorflow as tf
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR, labels='inferred', label_mode='binary', image_size=(64,64),
    batch_size=8, shuffle=True, validation_split=0.3, subset='validation', seed=123)
for X,y in val_ds.take(1):
    print("Validation batch:", X.shape, y.shape)